In [1]:
%conda install -c stefanotrv simple_icd_10 

^C

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Lab events dataframe

In [3]:
lab_events_df = pd.read_csv('data/LABEVENTS_HPO.csv')

In [4]:
lab_events_df

,row_id,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,flag,possible_hpo_features,active_hpo_features
0,6244563,10006,NaN,50868,2164-09-24 20:21:00,19,19.00,mEq/L,NaN,HP:0031963;HP:0031961;HP:0031962,NaN
1,6244564,10006,NaN,50882,2164-09-24 20:21:00,27,27.00,mEq/L,NaN,HP:0032066;HP:0032065;HP:0032067,NaN
2,6244565,10006,NaN,50893,2164-09-24 20:21:00,10.0,10.00,mg/dL,NaN,HP:0002901;HP:0004363;HP:0003072,NaN
3,6244566,10006,NaN,50902,2164-09-24 20:21:00,97,97.00,mEq/L,NaN,HP:0003113;HP:0011422;HP:0011423,NaN
4,6244567,10006,NaN,50912,2164-09-24 20:21:00,7.0,7.00,mg/dL,abnormal,HP:0012101;HP:0012100;HP:0003259,HP:0012101;HP:0003259
...,...,...,...,...,...,...,...,...,...,...,...
67606,20452679,44228,103379.0,51250,2170-12-24 04:09:00,88,88.00,fL,NaN,HP:0025066;HP:0025065;HP:0005518,NaN
67607,20452680,44228,103379.0,51265,2170-12-24 04:09:00,595,595.00,K/uL,abnormal,HP:0001873;HP:0011873;HP:0001894,HP:0001873;HP:0001894
67608,20452681,44228,103379.0,51277,2170-12-24 04:09:00,14.5,14.50,%,NaN,HP:0031965;HP:0031965,NaN
67609,20452682,44228,103379.0,51279,2170-12-24 04:09:00,2.76,2.76,m/uL,abnormal,HP:0020060;HP:0020058;HP:0020059,HP:0020060;HP:0020059


In [5]:
lab_events_df = lab_events_df.dropna(subset=['active_hpo_features'])

In [6]:
lab_events_df = lab_events_df[['subject_id','active_hpo_features']]

In [7]:
lab_events_df = lab_events_df.groupby(['subject_id'])['active_hpo_features'].apply(';'.join).reset_index()

In [8]:
lab_events_df

,subject_id,active_hpo_features
0,10006,HP:0012101;HP:0003259;HP:0001943;HP:0003074;HP...
1,10011,HP:0031964;HP:0003073;HP:0012117;HP:0003282;HP...
2,10013,HP:0031956;HP:0003113;HP:0011423;HP:0003236;HP...
3,10017,HP:0011905;HP:0040217;HP:0001943;HP:0003074;HP...
4,10019,HP:0031964;HP:0003073;HP:0012117;HP:0003282;HP...
...,...,...
95,44083,HP:0031851;HP:0001899;HP:0020062;HP:0020063;HP...
96,44154,H;P;:;0;4;1;0;1;7;2;H;P;:;0;5;0;0;1;1;6;HP:000...
97,44212,HP:0003073;HP:0012117;HP:0031956;HP:0032066;HP...
98,44222,HP:0012101;HP:0003259;HP:0001943;HP:0003074;HP...


# Diagnose dataframe

In [9]:
diagnose_df = pd.read_csv('data/DIAGNOSE_ICD_hpo.csv')

In [10]:
diagnose_df

,row_id,subject_id,hadm_id,seq_num,icd9_code,icd10_codes,hpo_features
0,112344,10006,142345,1,99591,A419,HP:0100806
1,112345,10006,142345,2,99662,T827XXA,NaN
2,112346,10006,142345,3,5672,NaN,NaN
3,112347,10006,142345,4,40391,I120,HP:0004421;HP:0009741;HP:0004972;HP:0005117
4,112348,10006,142345,5,42731,I4891,NaN
...,...,...,...,...,...,...,...
1756,397673,44228,103379,7,1975,NaN,NaN
1757,397674,44228,103379,8,45182,I808,HP:0004418
1758,397675,44228,103379,9,99592,R6520,NaN
1759,397676,44228,103379,10,2449,NaN,NaN


In [11]:
diagnose_df = diagnose_df.dropna(subset=['hpo_features','icd10_codes'])

In [12]:
diagnose_df = diagnose_df[diagnose_df['seq_num'] == 1]

In [13]:
diagnose_df = diagnose_df[['subject_id','icd10_codes','hpo_features']]

In [14]:



joined_hpos = diagnose_df.groupby(['subject_id'])['hpo_features'].apply(';'.join).reset_index()
diagnose_df = pd.merge(joined_hpos, diagnose_df[['subject_id','icd10_codes']], on="subject_id") 

In [15]:
df = pd.merge(lab_events_df, diagnose_df, on="subject_id")

In [16]:
 hpo_vocab_df = pd.Series(', '.join(df['hpo_features'].to_list()), name='hpo_vocab')

In [17]:
 icd10_vocab_df = pd.Series(', '.join(df['icd10_codes'].to_list()), name='icd10_vocab')

In [18]:
hpo_vocab_df

0    HP:0100806, HP:0100806, HP:0100806, HP:0001342...
Name: hpo_vocab, dtype: object

In [19]:
import re
hpo_vocab = set(re.split(r'\s+|;|,', hpo_vocab_df.iloc[0]))
icd10_vocab = set(re.split(r'\s+|;|,', icd10_vocab_df.iloc[0]))

In [20]:
icd10_vocab;

In [21]:

df = df[df["icd10_codes"].str.contains(";")==False]
df

,subject_id,active_hpo_features,hpo_features,icd10_codes
0,10006,HP:0012101;HP:0003259;HP:0001943;HP:0003074;HP...,HP:0100806,A419
1,10013,HP:0031956;HP:0003113;HP:0011423;HP:0003236;HP...,HP:0100806,A419
2,10019,HP:0031964;HP:0003073;HP:0012117;HP:0003282;HP...,HP:0100806,A419
3,10026,HP:0410289;HP:0410288;HP:0003113;HP:0011423;HP...,HP:0001342,I619
5,10029,HP:0003236;HP:0012101;HP:0003259;HP:0001943;HP...,HP:0100806,A409
6,10036,H;P;:;0;4;1;0;1;7;2;HP:0031956;H;P;:;0;5;0;0;1...,HP:0100806,A419
7,10040,HP:0003073;HP:0012117;HP:0031956;HP:0003113;HP...,HP:0005117;HP:0004972;HP:0004421,I110
8,10042,HP:0003282;HP:0003155;HP:0031956;HP:0002904;HP...,HP:0000819;HP:0001997;HP:0100749;HP:0040270;HP...,I214
10,10045,HP:0032066;HP:0032067;HP:0002901;HP:0003072;HP...,HP:0100806,A414
11,10056,HP:0003073;HP:0012117;HP:0410289;HP:0410288;HP...,HP:0100806,A419


In [22]:
import numpy as np
train_size = 0.6
validate_size = 0.2
train_df, val_df, test_df = np.split(df.sample(frac=1), [int(train_size * len(df)), int((validate_size + train_size) * len(df))])

In [23]:
test_df

,subject_id,active_hpo_features,hpo_features,icd10_codes
21,10093,HP:0031963;HP:0031962;HP:0032066;HP:0032067;HP...,HP:0000819;HP:0001997;HP:0100749;HP:0040270;HP...,I214
7,10040,HP:0003073;HP:0012117;HP:0031956;HP:0003113;HP...,HP:0005117;HP:0004972;HP:0004421,I110
22,10094,HP:0003236;HP:0032232;HP:0012101;HP:0003259;HP...,HP:0100806;HP:0000845;HP:0000870;HP:0000280;HP...,A419
42,41976,HP:0002901;HP:0003072;HP:0002148;HP:0002905;HP...,HP:0100806;HP:0100806;HP:0100806;HP:0002090;HP...,J189
40,41976,HP:0002901;HP:0003072;HP:0002148;HP:0002905;HP...,HP:0100806;HP:0100806;HP:0100806;HP:0002090;HP...,A419
11,10056,HP:0003073;HP:0012117;HP:0410289;HP:0410288;HP...,HP:0100806,A419
20,10089,HP:0031956;HP:0032066;HP:0032067;HP:0003236;HP...,HP:0001342,I619
3,10026,HP:0410289;HP:0410288;HP:0003113;HP:0011423;HP...,HP:0001342,I619
30,10132,HP:0003282;HP:0003155;HP:0002901;HP:0003072;HP...,HP:0002090,B371


In [24]:
sum(df['hpo_features'].apply(lambda x: len(x)).tolist())/97

29.61855670103093

In [25]:
df['icd10_codes'] = df['icd10_codes'].astype(str)

In [33]:
import torch
import torch.nn as nn
import re

def convert(input, hpo_to_ix):
    idxs = []

    for w in input:
        if w in hpo_to_ix.keys():
          idxs.append(hpo_to_ix[w])




    return torch.tensor(idxs, dtype=torch.long)


def vec_to_word(input, ix_to_word):
    words = []

    for w in input:
        if w in ix_to_word.keys():
            words.append(ix_to_word[w])

    return words




CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMDEDDING_DIM = 100
OUTPUT_SIZE = 10



features,target = train_df['hpo_features'].copy(), train_df['icd10_codes'].copy()
val_features,val_target = val_df['hpo_features'].copy(), val_df['icd10_codes'].copy()
test_features,test_target = test_df['hpo_features'].copy(), test_df['icd10_codes'].copy()






# By deriving a set from `raw_text`, we deduplicate the array
#vocab = set(raw_text)
hpo_vocab_size = len(hpo_vocab)
icd10_vocab_size = len(icd10_vocab)


hpo_to_ix = {word:ix for ix, word in enumerate(hpo_vocab)}


ix_to_word = {ix:word for ix, word in enumerate(hpo_vocab)}

icd10_to_ix = {word:ix for ix, word in enumerate(icd10_vocab)}
ix_to_icd10 = {ix:word for ix, word in enumerate(icd10_vocab)}



class CBOW(torch.nn.Module):
    def __init__(self, hpo_vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(hpo_vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim,128)
        self.activation_function1 = nn.ReLU()
        
        #out: 1 x hpo_vocab_size
        self.linear2 = nn.Linear(128, icd10_vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)

        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([hpo_to_ix[word]])
        return self.embeddings(word).view(1,-1)


model = CBOW(hpo_vocab_size, EMDEDDING_DIM)

loss_function = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

correct_predicted_train = 0

#TRAINING
for epoch in range(100):
    train_loss = 0
    val_loss = 0

    correct_predicted_train = 0
    correct_predicted_val = 0


    for x, y in zip(features,target):

        input_vector = convert(re.split(r'\s+|;|,', x), hpo_to_ix) 
        logs = model(input_vector)
        train_loss += loss_function(logs, torch.tensor([icd10_to_ix[y]]))


        pred = [torch.argmax(logs).item()]
        pred_word = vec_to_word(pred,ix_to_icd10)
        correct_predicted_train +=  1 if pred_word[0] == y else 0


    with torch.no_grad():
        for x, y in zip(val_features, val_target):
            input_vector = convert(re.split(r'\s+|;|,', x), hpo_to_ix) 
            logs = model(input_vector)
            val_loss += loss_function(logs, torch.tensor([icd10_to_ix[y]]))

            pred = [torch.argmax(logs).item()]
            pred_word = vec_to_word(pred,ix_to_icd10)
            correct_predicted_val +=  1 if pred_word[0] == y else 0




    #optimize at the end of each epoch
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    print(" train accuracy: {}".format(correct_predicted_train/len(target)))
    print(" val accuracy: {}".format(correct_predicted_val/len(val_df)))





 train accuracy: 0.0
 val accuracy: 0.0
 train accuracy: 0.0
 val accuracy: 0.0
 train accuracy: 0.4
 val accuracy: 0.25
 train accuracy: 0.72
 val accuracy: 0.625
 train accuracy: 0.72
 val accuracy: 0.625
 train accuracy: 0.76
 val accuracy: 0.625
 train accuracy: 0.76
 val accuracy: 0.625
 train accuracy: 0.8
 val accuracy: 0.75
 train accuracy: 0.8
 val accuracy: 0.625
 train accuracy: 0.8
 val accuracy: 0.625
 train accuracy: 0.84
 val accuracy: 0.625
 train accuracy: 0.84
 val accuracy: 0.625
 train accuracy: 0.84
 val accuracy: 0.625
 train accuracy: 0.84
 val accuracy: 0.625
 train accuracy: 0.84
 val accuracy: 0.625
 train accuracy: 0.84
 val accuracy: 0.625
 train accuracy: 0.84
 val accuracy: 0.625
 train accuracy: 0.84
 val accuracy: 0.75
 train accuracy: 0.88
 val accuracy: 0.75
 train accuracy: 0.88
 val accuracy: 0.75
 train accuracy: 0.88
 val accuracy: 0.75
 train accuracy: 0.88
 val accuracy: 0.75
 train accuracy: 0.88
 val accuracy: 0.75
 train accuracy: 0.88
 val ac

In [36]:
import simple_icd_10 as icd



correct_predicted_test = 0

for x, y in zip(test_features, test_target):
    input_vector = convert(re.split(r'\s+|;|,', x), hpo_to_ix) 
    logs = model(input_vector)
    pred = [torch.argmax(logs).item()]
    pred_word = vec_to_word(pred,ix_to_icd10)

    print("predicted: {}".format(icd.get_description(pred_word[0])))

    correct_predicted_test +=  1 if pred_word[0] == y else 0

#print("test accuracy: {}".format(correct_predicted_test/len(test_df)))


predicted: Acute subendocardial myocardial infarction
predicted: Disease of pericardium, unspecified
predicted: Malignant neoplasm: Overlapping lesion of oesophagus
predicted: Sepsis, unspecified
predicted: Sepsis, unspecified
predicted: Sepsis, unspecified
predicted: Sepsis, unspecified
predicted: Sepsis, unspecified
predicted: Pneumonia, unspecified
